In [ ]:
!pip install -q langchain langchain_openai langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.7/360.7 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58

In [ ]:
from langchain_openai import ChatOpenAI
from langgraph.graph import Graph

# Set the model as ChatOpenAI
model = ChatOpenAI(model='meta-llama/llama-3.1-8b-instruct', temperature=0)


In [ ]:
def function_1(state):
    messages = state['messages']
    user_input = messages[-1]
    complete_query = """You are an English writer assistant. Find all the grammatical mistakes in the provided text by the user and write the correct version with explanation for each one
            \n Here is the user text: """ + user_input
    response = model.invoke(complete_query)
    state['messages'].append(response.content) # appending AIMessage response to the AgentState
    return state

def function_2(state):
    messages = state['messages']
    agent_response = messages[-1]
    complete_query = """Rewrite the text and correct all the mistakes also improve the quality of the text """ + agent_response
    response = model.invoke(complete_query)
    state['messages'].append(response.content) # appending AIMessage response to the AgentState
    return state

def function_3(state):
    messages = state['messages']
    user_input = messages[0]
    first_response = messages[1]
    second_response = messages[-1]
    output =  user_input + "\n " + first_response + "\n " + second_response
    return output


In [ ]:
workflow = Graph()

workflow.add_node("agent1", function_1)
workflow.add_node("agent2", function_2)
workflow.add_node("responder", function_3)

workflow.add_edge('agent1', 'agent2')
workflow.add_edge('agent2', 'responder')

workflow.set_entry_point("agent1")
workflow.set_finish_point("responder")

app = workflow.compile()

In [ ]:
inputs = {"messages": ["Helo am fine wat are you doin. Nice too met you"]}
app.invoke(inputs)

'Helo am fine wat are you doin. Nice too met you\n Here are the grammatical mistakes in the provided text with their corrections and explanations:\n\n**Original Text:** Helo am fine wat are you doin. Nice too met you\n\n**Corrected Text:** Hello, I\'m fine. What are you doing? Nice to meet you.\n\n**Explanation:**\n\n1. **"Helo" should be "Hello"**: The correct spelling of the greeting is "Hello", not "Helo".\n2. **"am" should be "I\'m"**: "I\'m" is a contraction of "I am", which is the correct way to express the sentence in informal writing. "Am" is a verb ending, but it\'s not used in this context.\n3. **"wat" should be "what"**: The correct spelling of the word is "what", not "wat".\n4. **"are you doin" should be "are you doing"**: The correct verb form is "doing", not "doin". "Doin" is a colloquial or informal way of writing, but it\'s not standard English.\n5. **"Nice too" should be "Nice to"**: The correct phrase is "Nice to", not "Nice too". "Too" is used to indicate also or ver

In [ ]:
input = {"messages": ["Helo am fine wat are you doin. Nice too met you"]}
for output in app.stream(input):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'agent1':
---
{'messages': ['Helo am fine wat are you doin. Nice too met you', 'Here are the grammatical mistakes in the provided text with their corrections and explanations:\n\n**Original Text:** Helo am fine wat are you doin. Nice too met you\n\n**Corrected Text:** Hello, I\'m fine. What are you doing? Nice to meet you.\n\n**Explanation:**\n\n1. **"Helo" should be "Hello"**: The correct spelling of the greeting is "Hello", not "Helo".\n2. **"am" should be "I\'m"**: "I\'m" is a contraction of "I am", which is the correct way to express the sentence in informal writing. "Am" is a verb ending, but it\'s not used as a standalone word in this context.\n3. **"wat" should be "what"**: The correct spelling of the word is "what", not "wat".\n4. **"are you doin" should be "are you doing"**: The correct verb form is "doing", not "doin". "Doin" is a colloquial or informal way of writing, but it\'s not standard English.\n5. **"Nice too" should be "Nice to"**: The correct phrase 